# Pt bulk and surface

In this tutorial, we'll start with a bulk platinum unit cell and end with a platinum surface. We'll start by relaxing the bulk Pt structure and extracting some data from the calculation. We'll then use that relaxed Pt unit cell to create a surface under solvent and bias.

To start, take the POSCAR_bulk file from the files/Pt_111 directory in the tutorial files. Make a new directory in you ```scractch/tutorial/``` directory called Pt. Copy the POSCAR_bulk file into that directory and rename it to POSCAR.

Like we did in the last tutorial, make the lattice and position files by running ```pos2jdftx.py```. Now, we need to build our in file. Start by opening a new in file in nano.

Start by typing
```bash
include init.ionpos
include init.lattice
````

Next, we need to set our k-point sampling. To calculate many important parameters (electron density, density of states...) it is necessary to integrate over all k in the Brillouin zone. This integral is instead approximated with a sum over a discrete set of k-points. There are different ways to create this mesh, but we'll be using a Monkhorst Pack, which evenly distributes the points within the reciprocal lattice:
<img src="../Pictures/JDFTx/Monkhorst_Pack.png" alt="Monkhorst Pack">
[Image source](https://wiki.bnl.gov/CFN-Computation/images/3/34/K-point.pdf)

In general, larger unit cells require fewer k-points. In our case, we're going to use an 8x8x8 k-point mesh. To set up the k-point sampling in the in file, type
```bash
kpoint-folding 8 8 8
```

We need to specify our psuedopotentials again, so make sure to include these lines in your in file
```bash
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.2.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.4.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.5.uspp
```

And include the cutoff energy tag:
```bash
elec-cutoff 20 100
```

We also want to optimize the geometry of our unit cell, so we have to add ionic-minimize tags. We're also going to use a new type of electronic state convergence algorithm called *Self Consistent Field* or *SCF* for short. SCF is an iterative method for solving the schroedinger equation for many electrons. SCF is the fastest method for electronic structure optimization in JDFTx, but we won't use it for bias calculations due to a convergence issue. For this solid metal calculation, it should not have that convergence issue.

Add these lines to the in file
```bash
electronic-SCF                   #Electronic state optimization
lattice-minimize nIterations 10
```

Now we'll specify our output variables. In this case, we want to see how the nuclear positions change after each ionic step. To get the state after each ionic step, use the tag ```dump Ionic State```. Add these lines to your in file
```bash
dump-name Pt.$VAR
dump End ElecDensity
dump End Ecomponents
dump Ionic State 
```

We still need a submission script. enter the following lines into a ```submit.sh``` file:
```bash
#!/bin/bash
#SBATCH -J Pt_111
#SBATCH --time=23:00:00
#SBATCH --tasks 4
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --account=ucb-general
#SBATCH --partition amilan
#SBATCH --qos=normal

export I_MPI_FABRICS=shm
module load intel impi

mpirun -np 4 /home/nisi6161/jdftx_alpine/build/jdftx -i Pt.in -o out
exit 0
```

You'll notice some differences from our N2 submission script. First, our job name was changed to ```Pt_111```. This time we also specified 4 tasks. Since our calculation includes multiple k-points, JDFTx is able to run them all in parallel. So by specifying 4 tasks, we ask Alpine to give us 4 cpus to parallelize our k-points across. In the mpirun command, you'll notice the ```-np 4``` tag, which tells the MPI parallelization program to make four parallel instances. JDFTx will then use those to run the k-points in parallel.

Use ```sbatch submit.sh``` to submit your script

### Analysis

We can check to see that the JDFTx calculation finished by typing ```tail -n 40 out```. As long as it finished normally, you can continue to the rest of the analysis.

You should see these files in your Pt calculation directory:
```bash
init.ionpos
init.lattice
out
POSCAR
Pt.Ecomponents
Pt.in
Ot.ionpos
Pt.lattice
Pt.n
Pt.wfns
submit.sh
```
Plus a slurm output file

We are going to do a few pieces of analysis on this calculation. First, let's look at the final structure. To do this, first activate the jdft conda environment
```conda activate jdft```

Then, run ```j2pos.py``` to turn the Pt.ionpos file into a CONTCAR

Visualize the CONTCAR in VESTA. It should look like this:

<img src="../Pictures/JDFTx/Pt_Relaxed.png" alt="Pt Relaxed">

This unit cell should look pretty strange to you compared to our starting unit cell, which looked like this:

<img src="../Pictures/JDFTx/Pt_Initial.png" alt="Pt Initial">

Did we take away Pt atoms from our unit cell? No. We actually just broke the perfect cubic symmetry which prevents VESTA from showing all atoms on the corners/faces. However, our actual geometry change was small. If you compare the CONTCAR and POSCAR, you'll see that the crystal dimensio0ns changed slightly, and the atom positions shifted slightly.

Now let's visualize the electron density around the lattice atoms. The electron density is written in the ```Pt.n``` file. However, this file isn't human readable, so we need to use a JDFTx helper script to extract it. Run these lines:
```bash
conda activate jdft #need our octave anaconda environment loaded
createXSF out dens.xsf Pt.n #create xsf density file
```

You should notice a new ```dens.xsf``` file in the calculation directory. Drag this file into VESTA to visualize it.

<img src="../Pictures/JDFTx/Pt_Density.png" alt="Pt Density">

The colored surfaces are called isosurfaces, and they represent the electron density distributed throughout the cell. You can see that the electrons most densly occupy the sphere surrounding each Pt atom. An inorganic chemist might point out that the spherical enelctron density looks odd. I agree! I'm not fully sure how to get an electron density that more resembles a d-orbital, but I think [Wannier orbitals](https://jdftx.org/SeparatedBands.html) could be the answer.

## Surfaces

We now have a converged bulk Pt structure. Since chemical reactions happen at the surface of a catalyst, we will need to turn this bulk structure into a surface. There are many ways to make surfaces, but we're going to use Pymatgen to do it.

In the same ```Pt``` calculation directroy we've been working in, we're going to call the ```surface_maker.py``` script to cut the bulk structure into surfaces. We specify the gemoetry of our surface cut using [Miller Indices](https://web.iit.edu/sites/web/files/departments/academic-affairs/academic-resource-center/pdfs/Miller_Indices.pdf). In this case, we are only going to look at the most common low index factes (100) and (111). Use the following syntax to cut the (111) facet:
```bash
surface_maker.py -i 111
```

The script should have printed a message to the command line. If you ```ls```, you should see that a new directory called ```111``` was created.

<img src="../Pictures/JDFTx/111_Surface.png" alt="111 Surface">

```cd``` into that directory and visualize the ```POSCAR_00``` file in VESTA.

<img src="../Pictures/JDFTx/Pt_Surface.png" alt="Pt Surface">

At the moment, this file looks very strange. It looks nothing like any of the unit cells we've worked with so far, and it's hard to see that it's actually a surface. However, if you set the boundaries to 3x3x1 in VESTA and set it to Space-filling, it is clear that our unit cell is indeed a surface:

<img src="../Pictures/JDFTx/Surface_Boundary.png" alt="Surface Boundary">

At first glance, this super cell looks like a good surface to throw into a DFT calculation. However, it has 54 atoms, which will significantly decrease the speed of our DFT calculation. Instead of using the 3x3x1 cell, we're going to make a 2x2x1 cell using the ```make_sueprcell.py``` script on Alpine.

Make sure you're in the ```111``` folder in the calculation directory, and call
```bash
make_supercell.py 221 -f POSCAR_00
```
You should now see a file called ```POSCAR_00_new```. You can visualize it in VESTA if you'd like to prove to yourself that we made a supercell of the (111) facet surface.

Now that your file is made, make a new directory called ```Pt_111``` outside of the ```Pt``` directory we've been working in.
Inside the tutorial directory you should now have three directories:
```bash
N2/ Pt/ Pt_111/
```
Copy your ```POSCAR_00_new``` file into the new ```Pt_111``` directory and rename it ```POSCAR```.

#### Your turn

Now, make a new (100) surface using the same methodology above. Make sure you cut the surface along the (100) facet and then make a 2x2x1 supercell. Copy that supercell into a Pt_100 directory next to the Pt_111 directory. If you get stuck and are unable to make the surface, both of the POSCARs can be found in this tutorial's Files/ directory.

If you succesfully made your POSCAR files, you're ready to build the in file and submit them to Alpine. We'll go through building the in file for the ```Pt_100``` system and then you'll have to build your own for ```Pt_111```.

First, start by running ```pos2jdftx.py``` in the ```Pt_100``` calculation directory. This should populate the directory with the ```init.ionpos``` and ```init.lattice``` files.

Let's start by including those in the in file. use nano to open a new file called ```in``` and add the lines:
```bash
include init.ionpos
include init.lattice
```

We're going to now specify how many bands we want JDFTx to calculate. Many of these bands will be low lying bands that don't participate in the surface chemistry we're interested in, and some will be very high energy bands that also won't matter. However, to make sure we get enough of the bands we care about, we're going to let ASE suggest how many bands to calculate. The group already has a script for doing this called ```suggest_input_tags.py``` which will estimate the k-point mesh and the number of bands we need based on the POSCAR alone.

Run ```suggest_input_tags.py``` in the calculation directory:

<img src="../Pictures/JDFTx/Suggest_Tags.png" alt="Suggest Tags">

The script gave us the tags we'll need to enter into our in file. Enter them in this format:
```bash
elec-n-bands 230
kpoint-folding 5 5 1
```